# 3️⃣ Combining Pre-Trained Adapters using AdapterFusion

In [the previous notebook](https://colab.research.google.com/drive/1ovA1_ENGU1TT4T6nz2bW2bzq8-Lg8mMW?usp=sharing), we loaded a single pre-trained adapter from _AdapterHub_. Now we will explore how to take advantage of multiple pre-trained adapters to combine their knowledge on a new task. This setup is called **AdapterFusion** ([Pfeiffer et al., 2020](https://arxiv.org/pdf/2005.00247.pdf)).

For this guide, we select **CommitmentBank** ([De Marneffe et al., 2019](https://github.com/mcdm/CommitmentBank)), a three-class textual entailment dataset, as our target task. We will fuse [adapters from AdapterHub](https://adapterhub.ml/explore/) which were pre-trained on different tasks. During training, their represantions are kept fix while a newly introduced fusion layer is trained. As our base model, we will use BERT (`bert-base-uncased`). 

## Installation

Again, we install `adapter-transformers` and HuggingFace's `datasets` library first:

In [ ]:
!pip install git+https://github.com/Adapter-Hub/adapter-transformers.git
!pip install datasets

  Cloning https://github.com/Adapter-Hub/adapter-transformers.git to /tmp/pip-req-build-1z8tpz45
  Running command git clone -q https://github.com/Adapter-Hub/adapter-transformers.git /tmp/pip-req-build-1z8tpz45
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for adapter-transformers: filename=adapter_transformers-1.1.0-cp36-none-any.whl size=1326313 sha256=323e3925c7d5ffd29965221560a53e985affd4cc5f7227654c84c363386a0f87
  Stored in directory: /tmp/pip-ephem-wheel-cache-_u4tby_0/wheels/b0/56/c9/5bf1c51cd513412090ad751ab10fc025210176bf0a82dd8af3
Successfully built adapter-transformers


## Dataset Preprocessing

Before setting up training, we first prepare the training data. CommimentBank is part of the SuperGLUE benchmark and can be loaded via HuggingFace `datasets` using one line of code:

In [ ]:
from datasets import load_dataset

dataset = load_dataset("super_glue", "cb")
dataset.num_rows

Reusing dataset super_glue (/root/.cache/huggingface/datasets/super_glue/cb/1.0.2/41d9edb3935257e1da4b7ce54cd90df0e8bb255a15e46cfe5cbc7e1c04f177de)


{'test': 250, 'train': 250, 'validation': 56}

Every dataset sample has a premise, a hypothesis and a three-class class label:

In [ ]:
dataset['train'].features

{'hypothesis': Value(dtype='string', id=None),
 'idx': Value(dtype='int32', id=None),
 'label': ClassLabel(num_classes=3, names=['entailment', 'contradiction', 'neutral'], names_file=None, id=None),
 'premise': Value(dtype='string', id=None)}

Now, we need to encode all dataset samples to valid inputs for our `bert-base-uncased` model. Using `dataset.map()`, we can pass the full dataset through the tokenizer in batches:

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def encode_batch(batch):
  """Encodes a batch of input data using the model tokenizer."""
  return tokenizer(
      batch["premise"],
      batch["hypothesis"],
      max_length=180,
      truncation=True,
      padding="max_length"
  )

# Encode the input data
dataset = dataset.map(encode_batch, batched=True)
# The transformers model expects the target class column to be named "labels"
dataset.rename_column_("label", "labels")
# Transform to pytorch tensors and only output the required columns
dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

Loading cached processed dataset at /root/.cache/huggingface/datasets/super_glue/cb/1.0.2/41d9edb3935257e1da4b7ce54cd90df0e8bb255a15e46cfe5cbc7e1c04f177de/cache-90a019abe1b68c76.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/super_glue/cb/1.0.2/41d9edb3935257e1da4b7ce54cd90df0e8bb255a15e46cfe5cbc7e1c04f177de/cache-bea936ff6bf18563.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/super_glue/cb/1.0.2/41d9edb3935257e1da4b7ce54cd90df0e8bb255a15e46cfe5cbc7e1c04f177de/cache-1247294aea18cc08.arrow


New we're ready to setup AdapterFusion...

## Fusion Training

We use a pre-trained BERT model from HuggingFace and instantiate our model using `BertModelWithHeads`.

In [ ]:
from transformers import BertConfig, BertModelWithHeads

id2label = {id: label for (id, label) in enumerate(dataset["train"].features["labels"].names)}

config = BertConfig.from_pretrained(
    "bert-base-uncased",
    id2label=id2label,
)
model = BertModelWithHeads.from_pretrained(
    "bert-base-uncased",
    config=config,
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModelWithHeads: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModelWithHeads from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Now we have everything set up to load our _AdapterFusion_ setup. First, we load three adapters pre-trained on different tasks from the Hub: MultiNLI, QQP and QNLI. As we don't need their prediction heads, we pass `with_head=False` to the loading method. Next, we add a new fusion layer that combines all the adapters we've just loaded. Finally, we add a new classification head for our target task on top.

In [ ]:
from transformers import AdapterType

# Load the pre-trained adapters we want to fuse
model.load_adapter("nli/multinli@ukp", AdapterType.text_task, load_as="multinli", with_head=False)
model.load_adapter("sts/qqp@ukp", AdapterType.text_task, with_head=False)
model.load_adapter("nli/qnli@ukp", AdapterType.text_task, with_head=False)
# Add a fusion layer for all loaded adapters
model.add_fusion(["multinli", "qqp", "qnli"])

# Add a classification head for our target task
model.add_classification_head("cb", num_labels=len(id2label))

The last preparation step is to define and activate our adapter setup. Similar to `train_adapter()`, `train_fusion()` does two things: It freezes all weights of the model (including adapters!) except for the fusion layer and classification head. It also activates the given adapter setup to be used in very forward pass.

The syntax for the adapter setup (which is also applied to other methods such as `set_active_adapters()`) works as follows:

- a single string is interpreted as a single adapter
- a list of strings is interpreted as a __stack__ of adapters
- a _nested_ list of strings is interpreted as a __fusion__ of adapters

In [ ]:
# Unfreeze and activate fusion setup
adapter_setup = [
  ["multinli", "qqp", "qnli"]
]
model.train_fusion(adapter_setup)

For training, we make use of the `Trainer` class built-in into `transformers`. We configure the training process using a `TrainingArguments` object and define a method that will calculate the evaluation accuracy in the end. We pass both, together with the training and validation split of our dataset, to the trainer instance.

In [ ]:
import numpy as np
from transformers import TrainingArguments, Trainer, EvalPrediction

training_args = TrainingArguments(
    learning_rate=5e-5,
    num_train_epochs=5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_steps=200,
    output_dir="./training_output",
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
)

def compute_accuracy(p: EvalPrediction):
  preds = np.argmax(p.predictions, axis=1)
  return {"acc": (preds == p.label_ids).mean()}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    compute_metrics=compute_accuracy,
)

Start the training 🚀 (this will take a while)

In [ ]:
trainer.train()

/usr/local/lib/python3.6/dist-packages/datasets/arrow_dataset.py:851: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.tensor(x, **format_kwargs)


Step,Training Loss


After completed training, let's check how well our setup performs on the validation set of our target dataset:

In [ ]:
trainer.evaluate()

We can also use our setup to make some predictions (the example is from the test set of CB):

In [ ]:
import torch

def predict(premise, hypothesis):
  encoded = tokenizer(premise, hypothesis, return_tensors="pt")
  logits = model(**encoded)[0]
  pred_class = torch.argmax(logits).item()
  return id2label[pred_class]

predict("""
``It doesn't happen very often.'' Karen went home
happy at the end of the day. She didn't think that
the work was difficult.
""",
"the work was difficult"
)

Finally, we can extract and save our fusion layer as well as all the adapters we used for training. Both can later be reloaded into the pre-trained model again.

In [ ]:
model.save_adapter_fusion("./saved", "multinli,qqp,qnli")
model.save_all_adapters("./saved")

!ls -l saved

That's it. Do check out [the paper on AdapterFusion](https://arxiv.org/pdf/2005.00247.pdf) for a more theoretical view on what we've just seen.